In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Dropout, BatchNormalization, Input, Reshape, Lambda
import pandas as pd
import numpy as np
from pathlib import Path
import tkinter as tk
import os
import random
random.seed(42)

np.random.seed(42)
tf.random.set_seed(42)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
print("Dispositivos disponibles:", tf.config.list_physical_devices())

# Verificar si realmente estamos en CPU
if tf.config.list_physical_devices('GPU'):
    print("Advertencia: ¡Todavía hay una GPU activa!")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf
print("Dispositivos disponibles después de forzar CPU:", tf.config.list_physical_devices())


2025-02-10 19:43:29.929379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739213010.141132    7153 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739213010.201756    7153 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 19:43:30.626199: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Dispositivos disponibles después de forzar CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2025-02-10 19:43:35.075033: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-02-10 19:43:35.075109: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: pc1
2025-02-10 19:43:35.075127: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: pc1
2025-02-10 19:43:35.075303: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 535.230.2
2025-02-10 19:43:35.075363: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 535.230.2
2025-02-10 19:43:35.075377: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 535.230.2


In [ ]:
# Crear una ventana oculta de Tkinter
from tkinter import filedialog

# Create a hidden Tkinter window
root = tk.Tk()
root.withdraw()  # Hide the main window

# Initial path
initial_path = Path(r'/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test')


try:
    # Select the first path
    print("Select the path for 'x_train'")
    x_train_path = Path(filedialog.askdirectory(
        title="Select the folder for 'x_train'",
        initialdir=initial_path
    ))

    # Select the second path
    print("Select the path for 'y_train'")
    y_train_path = Path(filedialog.askdirectory(
        title="Select the folder for 'y_train'",
        initialdir=initial_path
    ))

    # Select the third path
    print("Select the path for 'x_val'")
    x_val_path = Path(filedialog.askdirectory(
        title="Select the folder for 'x_val'",
        initialdir=initial_path
    ))

    # Select the fourth path
    print("Select the path for 'y_val'")
    y_val_path = Path(filedialog.askdirectory(
        title="Select the folder for 'y_val'",
        initialdir=initial_path
    ))

    # Show the selected paths
    print(f"Path selected for x_train: {x_train_path}")
    print(f"Path selected for y_train: {y_train_path}")
    print(f"Path selected for x_val: {x_val_path}")
    print(f"Path selected for y_val: {y_val_path}")

except Exception as e:
    print(f"Error selecting paths: {e}")

finally:
    root.destroy()  # Ensure the Tkinter window is closed

In [ ]:
# Función para cargar y procesar los datos
def load_data(x_path, y_path):
    X_data, Y_data = [], []
    num_points_list = []

    for x_file in Path(x_path).iterdir():
        if x_file.is_file() and x_file.suffix == '.csv':
            y_file = Path(y_path) / x_file.name.replace('x_', 'y_')

            x_df = pd.read_csv(x_file, header=None, skiprows=1)
            y_df = pd.read_csv(y_file, header=None, skiprows=1)

            x_features = x_df.iloc[:, [0, 1, 2]].astype(float).values
            labels = y_df.iloc[:, 0].values

            X_data.append(x_features)
            Y_data.append(labels)
            num_points_list.append(len(x_features))
    
    return X_data, Y_data, min(num_points_list)  # Retorna además el número mínimo de puntos

# Cargar datos y obtener la cantidad mínima de puntos
x_train, y_train, min_train_points = load_data(x_train_path, y_train_path)
x_val, y_val, min_val_points = load_data(x_val_path, y_val_path)

min_points = min(min_train_points, min_val_points)  # Definir la cantidad mínima global

# def downsample_data(X_data, Y_data, num_points):
#    X_downsampled, Y_downsampled = [], []
#    for x, y in zip(X_data, Y_data):
#        indices = np.random.permutation(len(x))[:num_points]  # Selección aleatoria
#       X_downsampled.append(x[indices])
#        Y_downsampled.append(y[indices])
#    return np.array(X_downsampled), np.array(Y_downsampled)

# Aplicar el downsampling
#x_train, y_train = downsample_data(x_train, y_train, min_points)

#x_val, y_val = downsample_data(x_val, y_val, min_points)

In [ ]:
# Verificar las formas de los datos
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")

In [ ]:
# Definir parámetros
num_features = 5  # XYZ + remission + otra característica
num_classes = 64  # Número de clases a segmentar

# Función T-Net para la transformación de características
def tnet(inputs, k):
    x = Conv1D(64, kernel_size=1, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv1D(128, kernel_size=1, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv1D(256, kernel_size=1, activation='relu')(x)
    x = BatchNormalization()(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(k * k, activation='linear', kernel_initializer='zeros')(x)
    x = Reshape((k, k))(x)
    return x

# Entrada con tamaño variable
inputs = Input(shape=(None, num_features))

# Aplicar T-Net a la entrada
tnet_transform = tnet(inputs, num_features)
transformed_inputs = Lambda(lambda x: tf.linalg.matmul(x, tnet_transform))(inputs)

# Bloque de extracción de características
x = Conv1D(64, kernel_size=1, activation='relu')(transformed_inputs)
x = BatchNormalization()(x)
x = Conv1D(128, kernel_size=1, activation='relu')(x)
x = BatchNormalization()(x)
x = Conv1D(256, kernel_size=1, activation='relu')(x)
x = BatchNormalization()(x)

# Aplicar T-Net en el espacio de características
feature_tnet = tnet(x, 64)
x = Lambda(lambda x: tf.linalg.matmul(x, feature_tnet))(x)

# Expansión de características para segmentación punto a punto
x = Conv1D(128, kernel_size=1, activation='relu')(x)
x = BatchNormalization()(x)
x = Conv1D(64, kernel_size=1, activation='relu')(x)
x = BatchNormalization()(x)

# Capa de salida con softmax para segmentación punto a punto
outputs = Conv1D(num_classes, kernel_size=1, activation='softmax')(x)

# Construcción del modelo funcional
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# Definir métricas adicionales
def mean_iou(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    return tf.keras.metrics.MeanIoU(num_classes=num_classes)(y_true, y_pred)

def dice_coefficient(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=-1), depth=num_classes)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2])
    return tf.reduce_mean(2.0 * intersection / (union + 1e-6))

In [ ]:
# Compilación del modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', mean_iou, dice_coefficient])

# Resumen del modelo
model.summary()